<h1>RECREATING THE MODEL</h1>

In [1]:
import pandas as pd

import pandas as pd

# df1 = dataframe of manually encoded datasets.

df1 = pd.read_csv('dataset-december.csv', encoding='utf-8')
df1 = df1.drop(columns = ['id'])
df1

,question,label
0,It is the sum of the terms of a sequence.,series_and_sequence
1,Find the sum of the first ten terms of the ari...,series_and_sequence
2,Find the sum of the first 25 terms of the arit...,series_and_sequence
3,The sum of the first 10 terms of an arithmetic...,series_and_sequence
4,The third term of an arithmetic sequence is −1...,series_and_sequence
...,...,...
840,Determine the zeros of the polynomial function...,polynomial_equation
841,Determine the zeros of the polynomial function...,polynomial_equation
842,Determine the zeros of the polynomial function...,polynomial_equation
843,Determine the zeros of the polynomial function...,polynomial_equation


In [2]:
df2 = pd.DataFrame(columns=['question', 'label'])
with open("algebra__polynomial_roots.txt", "r") as data:
    #First 109 instance of Dataset      
    list_item = data.readlines()[:217]
    for item in range(0,len(list_item),2):
        df2 = df2.append({"question":list_item[item], "label":"polynomial_equation"}, ignore_index=True)
    data.close()

initial_dataset = df1.append(df2, ignore_index=True)
initial_dataset.to_html('dataset_html.html')
print('DATASET PREVIEW')
print(initial_dataset)

DATASET PREVIEW
                                              question                label
0            It is the sum of the terms of a sequence.  series_and_sequence
1    Find the sum of the first ten terms of the ari...  series_and_sequence
2    Find the sum of the first 25 terms of the arit...  series_and_sequence
3    The sum of the first 10 terms of an arithmetic...  series_and_sequence
4    The third term of an arithmetic sequence is −1...  series_and_sequence
..                                                 ...                  ...
949                Factor -35*k**2 + 27415*k - 7830.\n  polynomial_equation
950       Factor -6*n**4/5 - 116*n**3/5 - 38*n**2/5.\n  polynomial_equation
951  What is w in 3*w**3 - 3561*w**2 + 1058505*w - ...  polynomial_equation
952    Find v, given that 12*v**2 - 585*v - 147 = 0.\n  polynomial_equation
953  Determine z so that 3*z**3 + 234*z**2/5 + 864*...  polynomial_equation

[954 rows x 2 columns]


In [3]:
print('DATASET INFORMATION')
print(initial_dataset.label.value_counts())

DATASET INFORMATION
series_and_sequence    318
polynomials            318
polynomial_equation    318
Name: label, dtype: int64


In [4]:
import re

def change_to_dollar(exp):
    exp = re.sub("\.\.\.|\\dots|…","x",exp)
    exp = re.sub("=","$ $",exp)
    return exp

for i in range(len(initial_dataset)):
    qs = initial_dataset.loc[i, 'question']
    fx = re.search('[$].+?[=].+?[$]',qs)
    len_check = len(re.findall('[$].+?[=].+?[$]', qs))
    iplus = 0
    while iplus<len_check:
        iplus += 1
        qs = qs[:fx.span()[0]] + " " + change_to_dollar(fx.group()) + " " + qs[fx.span()[1]:]
        #check for the next LaTex Enclosure if any         
        fx = re.search('[$].+?[=].+?[$]', qs)
    #the new question is added to index i       
    initial_dataset.loc[i,'question'] = qs
    

pattern = r"\\sqrt\{(.+?)\}"
replacement = r"(\1)^{\\frac{1}{2}}"

dataset_sympytolatex = initial_dataset.replace(pattern, replacement, regex=True)

print(dataset_sympytolatex)

                                              question                label
0            It is the sum of the terms of a sequence.  series_and_sequence
1    Find the sum of the first ten terms of the ari...  series_and_sequence
2    Find the sum of the first 25 terms of the arit...  series_and_sequence
3    The sum of the first 10 terms of an arithmetic...  series_and_sequence
4    The third term of an arithmetic sequence is −1...  series_and_sequence
..                                                 ...                  ...
949                Factor -35*k**2 + 27415*k - 7830.\n  polynomial_equation
950       Factor -6*n**4/5 - 116*n**3/5 - 38*n**2/5.\n  polynomial_equation
951  What is w in 3*w**3 - 3561*w**2 + 1058505*w - ...  polynomial_equation
952    Find v, given that 12*v**2 - 585*v - 147 = 0.\n  polynomial_equation
953  Determine z so that 3*z**3 + 234*z**2/5 + 864*...  polynomial_equation

[954 rows x 2 columns]


In [5]:
from latex2sympy2 import latex2sympy
import re

df = dataset_sympytolatex

# Creates a new dataframe for updated question with freeform formula
new_df = pd.DataFrame(columns=['question', 'label'] )

# looping through all data    
for i in range(len(df)):
    new_df.loc[i,'label'] = df.loc[i,'label']
    test = df.loc[i, 'question']
    #regex value for searching all characters from '$' to the next '$' symbol
    check = re.search('[$].+?[$]',test)
    #checks how many instance of latex enclosure are there in the question     
    check_len = len(re.findall('[$].+?[$]',test))
    iteration = 0
    while iteration<check_len:
        iteration += 1
        #concatenate the question string from all characters before formula + formula + all characters after formula
        # formula = re.sub(" ","",str(latex2sympy(r"{}".format(check.group()))))
        # the formula is a converted LaTex to Sympy then removed all white spaces
        # uncomment print function to see the problem index and the token to be processed
        #print("index", i)
        #print(check.group())
        test = test[:check.span()[0]] + " " + re.sub(" ","",str(latex2sympy(r"{}".format(check.group())))) + " " + test[check.span()[1]:]
        #check for the next LaTex Enclosure if any         
        check = re.search('[$].+?[$]',test)
    #the new question is added to index i       
    new_df.loc[i,'question'] = test

new_df.head()

,question,label
0,It is the sum of the terms of a sequence.,series_and_sequence
1,Find the sum of the first ten terms of the ari...,series_and_sequence
2,Find the sum of the first 25 terms of the arit...,series_and_sequence
3,The sum of the first 10 terms of an arithmetic...,series_and_sequence
4,The third term of an arithmetic sequence is −1...,series_and_sequence


In [6]:
new_df = new_df.replace(' \+ ', '+', regex=True)
new_df = new_df.replace(' - ', '-', regex=True)
# We remove the equal sign so that the sympy library would not automatically evaluate the expression
new_df = new_df.replace(' = ', ' ', regex=True)
new_df = new_df.replace('\n','', regex=True) #white space
new_df = new_df.replace('\?','', regex=True) #Question marks
new_df = new_df.replace('\'','', regex=True) #quotation marks
new_df = new_df.replace(',','', regex=True)  #comma
new_df = new_df.replace('[\.]','', regex=True) #periods
new_df = new_df.replace('\t', ' ', regex=True) #tabs
new_df = new_df.replace('[;]', ' ', regex=True) #tabs
new_df = new_df.replace(':', ' ', regex=True) #tabs
new_df = new_df.replace(r"\_\_+", "__", regex=True)
dataset_freeform = new_df

In [7]:
question_list = new_df.drop(columns=['label']).values.tolist()
# Uncomment 'question_list' to display all question
# question_list

In [8]:
from sympy import *
from sympy.parsing.sympy_parser import parse_expr
import re

def tokenize_formula(expression):
    try:
        return srepr(parse_expr(expression, evaluate=False))
    except:
        return expression

for i in range(len(question_list)):
    item_question = question_list[i][0]
    tokenize_question_item = item_question.split()
    for x in range(len(tokenize_question_item)):
        check = re.findall('[\+]|[\*]|[-]|[/]|[0-9]',tokenize_question_item[x])
        if len(tokenize_question_item[x]) == 1:
            check = True 
        if check:
             
            #print("index: ",i)
            #print(tokenize_question_item[x])
            text = re.sub("\s|[,]","",str(tokenize_formula(tokenize_question_item[x])))
            #print(text)
            text = re.sub("\d+","(d)",text)
            text = re.sub("[(]['].+?['][)]","(x)",text)
            text = re.sub('\s+', '', text)
            tokenize_question_item[x] = text

    question_list[i] = ' '.join(tokenize_question_item)

# Adding the correlated first item in pair for the correlated column

In [9]:
# Adding the correlated first item in pair for the correlated column
remove_words = pd.read_csv('remove_words_v1.csv')
words = remove_words['remove_words'].values.tolist()

In [10]:
import sklearn
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Adding remove words to english stopwords from nltk
words += stopwords.words('english')

vect = CountVectorizer(token_pattern = '[a-zA-Z0-9,=+-<>()_{}]+',stop_words = words, min_df=0)
vect.fit(question_list)

CountVectorizer(min_df=0,
                stop_words=['(l)', '(polynomial)', '(w)',
                            'add(mul(integer((d))pow(symbol(x)integer((d))))symbol(x)mul(integer(-(d))mul(integer((d))integer((d))pow(mul(integer((d))symbol(x))mul(integer((d))pow(integer((d))integer(-(d))))))))',
                            'add(mul(integer(-(d))integer((d))pow(integer((d))mul(integer((d))pow(integer((d))integer(-(d))))))in...
                            'add(symbol(x)mul(integer(-(d))mul(integer((d))integer((d))symbol(x))))',
                            'add(symbol(x)symbol(x))', 'advised', 'allowance',
                            'answered', 'any)', 'asked', 'bank', 'blank', 'box',
                            'buy', 'cardboard', 'centimeter', 'centimeters',
                            'chain', 'christmas', 'classmate', 'classroom',
                            'closely', 'cm', 'composed', 'consider', 'constant',
                            'construction', 'contains', ...],
           

In [11]:
print("Column Names and their id")
keys = list(vect.vocabulary_.keys())
values = list(vect.vocabulary_.values())

with pd.option_context('display.max_rows', None,):
    print((pd.DataFrame(values ,index = keys, columns = ['id'])).sort_values(by = ['id']))

Column Names and their id
                                                     id
(binomial)                                            0
(d)                                                   1
(d)+(d)+(d)+                                          2
(d)cm                                                 3
(d)m                                                  4
(d)n                                                  5
(d)nd                                                 6
(d)rd                                                 7
(d)s_(d)                                              8
(d)st                                                 9
(d)th                                                10
(h)                                                  11
(if                                                  12
-(d)                                                 13
-(d)+(-(d))+(d)+                                     14
__                                                   15
a_k                   

In [12]:
print("Total Columns")
print(len(vect.get_feature_names_out()))

Total Columns
591


In [13]:
bag_of_words = vect.transform(question_list)
format(bag_of_words.toarray())

'[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]]'

In [14]:
import scipy.sparse

bow_Matrix = pd.DataFrame.sparse.from_spmatrix(bag_of_words, columns = vect.get_feature_names_out())
X = bow_Matrix.values
y = new_df.loc[:,"label"].values
bow_Matrix.head()

,(binomial),(d),(d)+(d)+(d)+,(d)cm,(d)m,(d)n,(d)nd,(d)rd,(d)s_(d),(d)st,...,way,week,whatever,whati,whether,whose,within,would,write,x
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state= 42)

In [16]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100,random_state = 42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [17]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[80  1  0]
 [ 8 76  0]
 [ 1  0 73]]
                     precision    recall  f1-score   support

polynomial_equation       0.90      0.99      0.94        81
        polynomials       0.99      0.90      0.94        84
series_and_sequence       1.00      0.99      0.99        74

           accuracy                           0.96       239
          macro avg       0.96      0.96      0.96       239
       weighted avg       0.96      0.96      0.96       239



In [18]:
# import pickle
# ## Uncomment to produce the file
# ## save the model to disk
# model = 'rfa_model.sav'
# pickle.dump(classifier, open(model, 'wb'))
# myvectorizer = 'vectorizer.sav'
# pickle.dump(vect, open(myvectorizer, 'wb'))

In [19]:
accuracy_score(y_test, y_pred)

0.9581589958158996